In [6]:
isOpenset = False
isUnloadable = False
isDataloader = True

In [7]:
isLocalServer = True
sys_path_to_be_added = '/notebook/personal/ksuchoi216/FaceID_model_rebuild/'
config_path = './configs/config.json'

%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import os

def checkLocalServer(isLocalServer, sys_path_to_be_added=None):
    print(f'isLocalServer is {isLocalServer}')
    if isLocalServer is True:
        if sys_path_to_be_added not in sys.path:
            sys.path.insert(0, sys_path_to_be_added)
            os.chdir(sys_path_to_be_added)

checkLocalServer(isLocalServer, sys_path_to_be_added)

from utils import Config_Manager
cfgm = Config_Manager(config_path)
cfg = cfgm.get_cfg()
!pwd
!nvidia-smi

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
isLocalServer is True
/notebook/personal/ksuchoi216/FaceID_model_rebuild
Fri Oct  7 14:15:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:04:00.0 Off |                    0 |
| N/A   33C    P0    33W / 250W |    847MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+------

In [8]:
from data_preprocessing import FolderDataset, DatasetFromNumpy
from data_preprocessing import buildDataLoaders, saveDataloaders
cfg["FolderDataset"]["folder_for_source"] = cfg["folder_for_source"]
folderDataset = FolderDataset(cfg["FolderDataset"])
folderDataset.setFilePath(
    './data',
    'extracted_np',
    'face_data',
    '.npy'
)

if isUnloadable:
    np_emb, np_lb = folderDataset.createNumpyData()
    folderDataset.saveToNumpy(np_emb, np_lb)
np_emb, np_lb = folderDataset.loadFromNumpy()

if isOpenset:
    kkc = [0, 1, 2]
    uuc = [3]
    folderDataset.createOpensetDataset(np_emb, np_lb, kkc, uuc)
else:
    print("creat a dataset...")
    dataset = DatasetFromNumpy(np_emb, np_lb)


device is cuda:0
image size: 240
Loading faces from ./source/faces_from_photos
0 jhoh
1 jhongyoo
2 jhyoo
3 kschoi
data length: 399
path_emb: ./data/extracted_np/face_data_emb.npy
path_lb: ./data/extracted_np/face_data_lb.npy
loaded emb:(399, 512) lb:(399,)
creat a dataset...
dataset initial emb:(399, 512) lb:(399,)


In [9]:
if isDataloader:
    dataloaders = buildDataLoaders(dataset, 4, 0.7, 0.2)
    saveDataloaders(source_folder='data',
                    save_folder='dataloaders_org',
                    dataloaders=dataloaders)

dataset length: (399) = tr (279) + val (79) + tt (41)
saved in ./data/dataloaders_org/dataloader_train.pt
saved in ./data/dataloaders_org/dataloader_val.pt
saved in ./data/dataloaders_org/dataloader_test.pt
